In [1]:
%cd C:/Users/sylee/Documents/GitHub/Final_2Kyoungh/data

C:\Users\sylee\Documents\GitHub\Final_2Kyoungh\data


In [2]:
import requests
import json
import pandas as pd

## 기본 세팅 부분
headers = {"X-Naver-Client-Id": "lJrKhZKFXCGXSoYzh7f7",
           "X-Naver-Client-Secret":"YYPQvTkBxR"}

url = "https://openapi.naver.com/v1/datalab/shopping/categories"
def change_index(df, column_name):
    df = df.set_index("period")
    df.columns = [column_name]
    return df


In [3]:
devices=['','pc','mo']

## 대분류 카테고리 설정
for i in devices:
    categories = [
        ## 가전, 전자
        {"name": "생활가전", "param": [ "50001412"]},
        {"name": "영상가전", "param": [ "50000208"]},
        {"name": "이미용가전", "param": [ "50000211"]},
        {"name": "주방가전", "param": [ "50000213"]},
        {"name": "계절가전", "param": [ "50000212"]},
        {"name": "카메라/캠코더용품", "param": [ "50000152"]},
        
        ## 컴퓨터 및 주변기기
        {"name": "PC", "param": [ "50000089"]},
        {"name": "PC부품", "param": [ "50000097"]},
        {"name": "PC액세서리", "param": [ "50000090"]},
        {"name": "저장장치", "param": [ "50000096"]},
        {"name": "멀티미디어장비", "param": [ "50000095"]},
        {"name": "모니터", "param": [ "50000153"]},
        {"name": "모니터주변기기", "param": [ "50000093"]},
        {"name": "노트북", "param": [ "50000151"]},
        {"name": "노트북액세서리", "param": [ "50000091"]},
        {"name": "소프트웨어", "param": [ "50000099"]},
        {"name": "주변기기", "param": [ "50000094"]},
        
        ## 통신기기
        {"name": "음향가전", "param": [ "50000209"]},
        {"name": "태블릿PC", "param": [ "50000152"]},
        {"name": "태블릿PC액세서리", "param": [ "50000092"]},
        
        ## 기타
        {"name": "자동차기기", "param": [ "50000214"]},
        {"name": "게임기/타이틀", "param": [ "50000088"]},        
        {"name": "광학기기/용품", "param": [ "50000207"]},        
        {"name": "학습기기", "param": [ "50000087"]}]
## API 요청 값 작성, 디지털/가전 = result[0], 패션의류 = result[1]
    data = {
    "startDate": "2021-05-10",
    "endDate": "2023-07-03",
    "timeUnit": "date",
    "category": [
      {"name": "디지털/가전", "param": [ "50000003"]},
      {"name": "휴대폰액세서리", "param": [ "50000205"]},
    ],
    "device":i,
    "gender":"",
    "ages":["30"]
    }
## 데이터 정제 및 merge 파트
    response = requests.post(url, headers=headers, json=data)

    results = response.json()["results"]


    df_1 = change_index(pd.DataFrame(results[0]["data"]), "디지털/가전")
    df_2 = change_index(pd.DataFrame(results[1]["data"]), "휴대폰액세서리")
##
    df_all_i = pd.merge(df_1, df_2, left_on="period", right_on="period")
## 반복문 통해서 카테고리에 설정된 값, 불러오고 저장, 합치기
    for category in categories:
        if len(data["category"]) > 2:
            data["category"].pop(2)
        data["category"].append(category)

        result = requests.post(url, headers=headers, json=data)

        results = result.json()["results"]

        df = change_index(pd.DataFrame(results[2]["data"]), category["name"])

        df_all_i = pd.merge(df_all_i, df, left_on="period", right_on="period")
    if i == "":
        df_all_1 = df_all_i
        df_all_1['유형']='PC+MO'
        df_all_1['성별']='전체'
        df_all_1['연령']='30'
    if i == "pc":
        df_all_2 = df_all_i
        df_all_2['유형']='PC'
        df_all_2['성별']='전체'
        df_all_2['연령']='30'
    if i == "mo":
        df_all_3 = df_all_i
        df_all_3['유형']='MO'
        df_all_3['성별']='전체'
        df_all_3['연령']='30'
##    
df_list=[df_all_1,df_all_2,df_all_3]
df_all_a=pd.concat(df_list)
df_all_a.drop(["디지털/가전"], axis=1, inplace=True)
df_all_a

,휴대폰액세서리,생활가전,영상가전,이미용가전,주방가전,계절가전,카메라/캠코더용품,PC,PC부품,PC액세서리,...,음향가전,태블릿PC,태블릿PC액세서리,자동차기기,게임기/타이틀,광학기기/용품,학습기기,유형,성별,연령
period,,,,,,,,,,,,,,,,,,,,,
2021-05-10,7.53385,0.31894,2.95236,2.34632,12.64393,7.50304,1.13355,0.51173,1.42602,0.63162,...,4.76890,1.13355,0.90748,1.19610,2.39852,0.06068,0.11692,PC+MO,전체,30
2021-05-11,7.27391,0.29513,2.41947,2.08737,10.80979,6.56972,0.93991,0.46756,1.43070,0.62255,...,4.42196,0.93991,0.85813,1.14415,2.05932,0.05940,0.10480,PC+MO,전체,30
2021-05-12,7.43388,0.27186,2.29367,1.97774,10.35957,7.24862,0.88963,0.43646,1.47112,0.61654,...,4.28512,0.88963,0.83945,1.07686,1.89704,0.05259,0.10243,PC+MO,전체,30
2021-05-13,6.88305,0.25140,2.06489,2.04374,9.75959,10.27666,0.81313,0.39224,1.29657,0.57577,...,3.94197,0.81313,0.75343,1.05645,1.77168,0.05131,0.09706,PC+MO,전체,30
2021-05-14,6.25906,0.24608,1.86933,1.91730,8.81198,12.42910,0.72385,0.37346,1.16534,0.51139,...,3.69239,0.72385,0.64463,0.96013,1.65165,0.04954,0.08123,PC+MO,전체,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-29,5.69128,0.18478,1.56630,1.67102,5.76583,8.67842,0.48245,0.18198,0.77871,0.42312,...,2.24758,0.48245,0.59100,0.61392,1.34712,0.04747,0.11658,MO,전체,30
2023-06-30,4.82716,0.17529,1.40904,1.51062,5.19613,7.22409,0.43913,0.18939,0.74067,0.38036,...,2.01970,0.43913,0.52655,0.61257,1.25975,0.04320,0.10753,MO,전체,30
2023-07-01,5.81123,0.17113,1.74069,1.73603,6.06226,8.27074,0.52841,0.18995,0.68263,0.36648,...,2.40602,0.52841,0.56622,0.61555,1.55388,0.05152,0.10618,MO,전체,30


In [4]:
devices=['','pc','mo']

## 대분류 카테고리 설정
for i in devices:
    categories = [
        ## 가전, 전자
        {"name": "생활가전", "param": [ "50001412"]},
        {"name": "영상가전", "param": [ "50000208"]},
        {"name": "이미용가전", "param": [ "50000211"]},
        {"name": "주방가전", "param": [ "50000213"]},
        {"name": "계절가전", "param": [ "50000212"]},
        {"name": "카메라/캠코더용품", "param": [ "50000152"]},
        
        ## 컴퓨터 및 주변기기
        {"name": "PC", "param": [ "50000089"]},
        {"name": "PC부품", "param": [ "50000097"]},
        {"name": "PC액세서리", "param": [ "50000090"]},
        {"name": "저장장치", "param": [ "50000096"]},
        {"name": "멀티미디어장비", "param": [ "50000095"]},
        {"name": "모니터", "param": [ "50000153"]},
        {"name": "모니터주변기기", "param": [ "50000093"]},
        {"name": "노트북", "param": [ "50000151"]},
        {"name": "노트북액세서리", "param": [ "50000091"]},
        {"name": "소프트웨어", "param": [ "50000099"]},
        {"name": "주변기기", "param": [ "50000094"]},
        
        ## 통신기기
        {"name": "음향가전", "param": [ "50000209"]},
        {"name": "태블릿PC", "param": [ "50000152"]},
        {"name": "태블릿PC액세서리", "param": [ "50000092"]},
        
        ## 기타
        {"name": "자동차기기", "param": [ "50000214"]},
        {"name": "게임기/타이틀", "param": [ "50000088"]},        
        {"name": "광학기기/용품", "param": [ "50000207"]},        
        {"name": "학습기기", "param": [ "50000087"]}]
## API 요청 값 작성, 디지털/가전 = result[0], 패션의류 = result[1]
    data = {
    "startDate": "2021-05-10",
    "endDate": "2023-07-03",
    "timeUnit": "date",
    "category": [
      {"name": "디지털/가전", "param": [ "50000003"]},
      {"name": "휴대폰액세서리", "param": [ "50000205"]},
    ],
    "device":i,
    "gender":"f",
    "ages":["30"]
    }
## 데이터 정제 및 merge 파트
    response = requests.post(url, headers=headers, json=data)

    results = response.json()["results"]


    df_1 = change_index(pd.DataFrame(results[0]["data"]), "디지털/가전")
    df_2 = change_index(pd.DataFrame(results[1]["data"]), "휴대폰액세서리")
##
    df_all_i = pd.merge(df_1, df_2, left_on="period", right_on="period")
## 반복문 통해서 카테고리에 설정된 값, 불러오고 저장, 합치기
    for category in categories:
        if len(data["category"]) > 2:
            data["category"].pop(2)
        data["category"].append(category)

        result = requests.post(url, headers=headers, json=data)

        results = result.json()["results"]

        df = change_index(pd.DataFrame(results[2]["data"]), category["name"])

        df_all_i = pd.merge(df_all_i, df, left_on="period", right_on="period")
    if i == "":
        df_all_1 = df_all_i
        df_all_1['유형']='PC+MO'
        df_all_1['성별']='여성'
        df_all_1['연령']='30'
    if i == "pc":
        df_all_2 = df_all_i
        df_all_2['유형']='PC'
        df_all_2['성별']='여성'
        df_all_2['연령']='30'
    if i == "mo":
        df_all_3 = df_all_i
        df_all_3['유형']='MO'
        df_all_3['성별']='여성'
        df_all_3['연령']='30'
##    
df_list=[df_all_1,df_all_2,df_all_3]
df_all_b=pd.concat(df_list)
df_all_b.drop(["디지털/가전"], axis=1, inplace=True)
df_all_b

,휴대폰액세서리,생활가전,영상가전,이미용가전,주방가전,계절가전,카메라/캠코더용품,PC,PC부품,PC액세서리,...,음향가전,태블릿PC,태블릿PC액세서리,자동차기기,게임기/타이틀,광학기기/용품,학습기기,유형,성별,연령
period,,,,,,,,,,,,,,,,,,,,,
2021-05-10,7.07447,0.25060,1.80581,2.67124,13.89651,6.72152,0.75598,0.19246,0.23570,0.31767,...,3.34083,0.75598,0.81872,0.37807,1.15841,0.03656,0.12478,PC+MO,여성,30
2021-05-11,6.86840,0.22868,1.45919,2.40096,12.05725,6.03252,0.65953,0.19064,0.22339,0.32599,...,3.06232,0.65953,0.78336,0.38085,1.01499,0.03561,0.11603,PC+MO,여성,30
2021-05-12,7.01268,0.22149,1.37123,2.27323,11.50283,6.62941,0.57339,0.15892,0.23500,0.33154,...,2.93892,0.57339,0.75295,0.36118,0.91334,0.03006,0.10970,PC+MO,여성,30
2021-05-13,6.64292,0.19072,1.29818,2.44611,10.74390,9.39649,0.53943,0.15979,0.21169,0.31239,...,2.74533,0.53943,0.69012,0.39878,0.82166,0.02807,0.10069,PC+MO,여성,30
2021-05-14,6.04335,0.19142,1.15390,2.34619,9.78332,11.60993,0.46317,0.16499,0.17851,0.27400,...,2.46933,0.46317,0.60294,0.34341,0.75061,0.03223,0.08032,PC+MO,여성,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-29,5.26372,0.13174,1.00649,1.98216,5.94969,8.11726,0.29665,0.07226,0.13136,0.27391,...,1.52200,0.29665,0.55272,0.18530,0.57245,0.02725,0.12450,MO,여성,30
2023-06-30,4.29463,0.12911,0.89542,1.76632,5.34472,6.64684,0.28190,0.06662,0.10712,0.24140,...,1.32701,0.28190,0.46786,0.21048,0.56503,0.02715,0.11962,MO,여성,30
2023-07-01,5.38081,0.12338,1.10797,1.98874,6.06085,7.35292,0.34646,0.07150,0.11398,0.24140,...,1.57030,0.34646,0.51278,0.20926,0.72393,0.03946,0.10298,MO,여성,30


In [5]:
devices=['','pc','mo']

## 대분류 카테고리 설정
for i in devices:
    categories = [
        ## 가전, 전자
        {"name": "생활가전", "param": [ "50001412"]},
        {"name": "영상가전", "param": [ "50000208"]},
        {"name": "이미용가전", "param": [ "50000211"]},
        {"name": "주방가전", "param": [ "50000213"]},
        {"name": "계절가전", "param": [ "50000212"]},
        {"name": "카메라/캠코더용품", "param": [ "50000152"]},
        
        ## 컴퓨터 및 주변기기
        {"name": "PC", "param": [ "50000089"]},
        {"name": "PC부품", "param": [ "50000097"]},
        {"name": "PC액세서리", "param": [ "50000090"]},
        {"name": "저장장치", "param": [ "50000096"]},
        {"name": "멀티미디어장비", "param": [ "50000095"]},
        {"name": "모니터", "param": [ "50000153"]},
        {"name": "모니터주변기기", "param": [ "50000093"]},
        {"name": "노트북", "param": [ "50000151"]},
        {"name": "노트북액세서리", "param": [ "50000091"]},
        {"name": "소프트웨어", "param": [ "50000099"]},
        {"name": "주변기기", "param": [ "50000094"]},
        
        ## 통신기기
        {"name": "음향가전", "param": [ "50000209"]},
        {"name": "태블릿PC", "param": [ "50000152"]},
        {"name": "태블릿PC액세서리", "param": [ "50000092"]},
        
        ## 기타
        {"name": "자동차기기", "param": [ "50000214"]},
        {"name": "게임기/타이틀", "param": [ "50000088"]},        
        {"name": "광학기기/용품", "param": [ "50000207"]},        
        {"name": "학습기기", "param": [ "50000087"]}]
## API 요청 값 작성, 디지털/가전 = result[0], 패션의류 = result[1]
    data = {
    "startDate": "2021-05-10",
    "endDate": "2023-07-03",
    "timeUnit": "date",
    "category": [
      {"name": "디지털/가전", "param": [ "50000003"]},
      {"name": "휴대폰액세서리", "param": [ "50000205"]},
    ],
    "device":i,
    "gender":"m",
    "ages":["30"]
    }
## 데이터 정제 및 merge 파트
    response = requests.post(url, headers=headers, json=data)

    results = response.json()["results"]


    df_1 = change_index(pd.DataFrame(results[0]["data"]), "디지털/가전")
    df_2 = change_index(pd.DataFrame(results[1]["data"]), "휴대폰액세서리")
##
    df_all_i = pd.merge(df_1, df_2, left_on="period", right_on="period")
## 반복문 통해서 카테고리에 설정된 값, 불러오고 저장, 합치기
    for category in categories:
        if len(data["category"]) > 2:
            data["category"].pop(2)
        data["category"].append(category)

        result = requests.post(url, headers=headers, json=data)

        results = result.json()["results"]

        df = change_index(pd.DataFrame(results[2]["data"]), category["name"])

        df_all_i = pd.merge(df_all_i, df, left_on="period", right_on="period")
    if i == "":
        df_all_1 = df_all_i
        df_all_1['유형']='PC+MO'
        df_all_1['성별']='남성'
        df_all_1['연령']='30'
    if i == "pc":
        df_all_2 = df_all_i
        df_all_2['유형']='PC'
        df_all_2['성별']='남성'
        df_all_2['연령']='30'
    if i == "mo":
        df_all_3 = df_all_i
        df_all_3['유형']='MO'
        df_all_3['성별']='남성'
        df_all_3['연령']='30'
##    
df_list=[df_all_1,df_all_2,df_all_3]
df_all_c=pd.concat(df_list)
df_all_c.drop(["디지털/가전"], axis=1, inplace=True)
df_all_c

,휴대폰액세서리,생활가전,영상가전,이미용가전,주방가전,계절가전,카메라/캠코더용품,PC,PC부품,PC액세서리,...,음향가전,태블릿PC,태블릿PC액세서리,자동차기기,게임기/타이틀,광학기기/용품,학습기기,유형,성별,연령
period,,,,,,,,,,,,,,,,,,,,,
2021-05-10,6.75112,0.33930,3.70334,1.59041,9.11584,7.07810,1.35333,0.77382,2.48533,0.86742,...,5.51803,1.35333,0.84978,1.88723,3.34640,0.07671,0.08838,PC+MO,남성,30
2021-05-11,6.47659,0.31749,3.05754,1.38804,7.60024,6.03667,1.08638,0.69084,2.50781,0.84086,...,5.15540,1.08638,0.79354,1.78424,2.85081,0.07529,0.07463,PC+MO,남성,30
2021-05-12,6.62642,0.28060,2.91179,1.31692,7.34088,6.68948,1.08392,0.66571,2.57286,0.82322,...,5.02719,1.08392,0.79089,1.67633,2.64986,0.06827,0.07700,PC+MO,남성,30
2021-05-13,5.97142,0.27491,2.55162,1.25471,7.01722,9.48644,0.97391,0.57970,2.26257,0.76576,...,4.57892,0.97391,0.69416,1.59591,2.50904,0.06799,0.07652,PC+MO,남성,30
2021-05-14,5.42719,0.26391,2.33332,1.12081,6.24548,11.20487,0.88562,0.53788,2.04645,0.68392,...,4.40083,0.88562,0.58027,1.47121,2.35589,0.06002,0.06837,PC+MO,남성,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-29,5.16232,0.21515,1.95581,0.98583,4.48093,7.76213,0.61913,0.28163,1.42093,0.52632,...,2.71394,0.61913,0.52814,1.02094,2.03927,0.06328,0.08551,MO,남성,30
2023-06-30,4.58435,0.19908,1.77223,0.92220,4.05939,6.59455,0.54912,0.30352,1.37304,0.47889,...,2.48852,0.54912,0.50089,0.98776,1.87098,0.05473,0.07308,MO,남성,30
2023-07-01,5.26323,0.19759,2.18818,1.10988,4.94842,7.86121,0.65197,0.29873,1.24705,0.45084,...,2.97710,0.65197,0.52677,0.99529,2.27484,0.05666,0.09053,MO,남성,30


In [6]:
df_list=[df_all_a,df_all_b,df_all_c]
df_all=pd.concat(df_list)
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7065 entries, 2021-05-10 to 2023-07-03
Data columns (total 28 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   휴대폰액세서리    7065 non-null   float64
 1   생활가전       7065 non-null   float64
 2   영상가전       7065 non-null   float64
 3   이미용가전      7065 non-null   float64
 4   주방가전       7065 non-null   float64
 5   계절가전       7065 non-null   float64
 6   카메라/캠코더용품  7065 non-null   float64
 7   PC         7065 non-null   float64
 8   PC부품       7065 non-null   float64
 9   PC액세서리     7065 non-null   float64
 10  저장장치       7065 non-null   float64
 11  멀티미디어장비    7065 non-null   float64
 12  모니터        7065 non-null   float64
 13  모니터주변기기    7065 non-null   float64
 14  노트북        7065 non-null   float64
 15  노트북액세서리    7065 non-null   float64
 16  소프트웨어      7065 non-null   float64
 17  주변기기       7065 non-null   float64
 18  음향가전       7065 non-null   float64
 19  태블릿PC      7065 non-null   float64
 20

In [7]:
## 엑셀에 저장
df_all.to_excel("MCLS_04_all_fm_30.xlsx", sheet_name="click ratio")
df_all.head()

,휴대폰액세서리,생활가전,영상가전,이미용가전,주방가전,계절가전,카메라/캠코더용품,PC,PC부품,PC액세서리,...,음향가전,태블릿PC,태블릿PC액세서리,자동차기기,게임기/타이틀,광학기기/용품,학습기기,유형,성별,연령
period,,,,,,,,,,,,,,,,,,,,,
2021-05-10,7.53385,0.31894,2.95236,2.34632,12.64393,7.50304,1.13355,0.51173,1.42602,0.63162,...,4.76890,1.13355,0.90748,1.19610,2.39852,0.06068,0.11692,PC+MO,전체,30
2021-05-11,7.27391,0.29513,2.41947,2.08737,10.80979,6.56972,0.93991,0.46756,1.43070,0.62255,...,4.42196,0.93991,0.85813,1.14415,2.05932,0.05940,0.10480,PC+MO,전체,30
2021-05-12,7.43388,0.27186,2.29367,1.97774,10.35957,7.24862,0.88963,0.43646,1.47112,0.61654,...,4.28512,0.88963,0.83945,1.07686,1.89704,0.05259,0.10243,PC+MO,전체,30
2021-05-13,6.88305,0.25140,2.06489,2.04374,9.75959,10.27666,0.81313,0.39224,1.29657,0.57577,...,3.94197,0.81313,0.75343,1.05645,1.77168,0.05131,0.09706,PC+MO,전체,30
2021-05-14,6.25906,0.24608,1.86933,1.91730,8.81198,12.42910,0.72385,0.37346,1.16534,0.51139,...,3.69239,0.72385,0.64463,0.96013,1.65165,0.04954,0.08123,PC+MO,전체,30
